In [1]:
# Data visualization
import numpy as np
import pandas as pd 

# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
import keras_tuner as kt

# Train-Test
from sklearn.model_selection import train_test_split
# Classification Report
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

import pickle

import warnings
warnings.filterwarnings('ignore')

## 1. Describe Dataset

In [2]:
# Determine important landmarks for plank
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_ELBOW",
    "RIGHT_ELBOW",
    "LEFT_WRIST",
    "RIGHT_WRIST",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
    "LEFT_HEEL",
    "RIGHT_HEEL",
    "LEFT_FOOT_INDEX",
    "RIGHT_FOOT_INDEX",
]

# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [4]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


# Remove duplicate rows (optional)
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''
    
    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_train.csv", sep=',', encoding='utf-8', index=False)


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))


df = describe_dataset("./train.csv")

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v', 'left_heel_x', 'left_heel_y', 'left_heel_z', 'left_heel_v', 'right_heel_x', 'right_heel_y', 'right_heel_z', 'right_heel_v', 'left_foot_index_x', 'lef

## 2. Preprocess Dataset

In [5]:
# load dataset
df = pd.read_csv("./train.csv")

# Categorizing label
df.loc[df["label"] == "C", "label"] = 0
df.loc[df["label"] == "H", "label"] = 1
df.loc[df["label"] == "L", "label"] = 2

print(f'Number of rows: {df.shape[0]} \nNumber of columns: {df.shape[1]}\n')
print(f"Labels: \n{df['label'].value_counts()}\n")

Number of rows: 28520 
Number of columns: 69

Labels: 
0    9904
2    9546
1    9070
Name: label, dtype: int64



In [6]:
# Standard Scaling of features
# Dump input scaler
with open("./model/input_scaler.pkl", "rb") as f2:
    sc = pickle.load(f2)

x = df.drop("label", axis = 1)
x = pd.DataFrame(sc.transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2, random_state=1234)

## 3. Build Model

### 3.1. Set up

In [8]:
stop_early = EarlyStopping(monitor='loss', patience=3)

# Final Results
final_models = {}

In [10]:
def describe_model(model):
    '''
    Describe Model architecture
    '''
    print(f"Describe models architecture")
    for i, layer in enumerate(model.layers):
        number_of_units = layer.units if hasattr(layer, 'units') else 0

        if hasattr(layer, "activation"):
            print(f"Layer-{i + 1}: {number_of_units} units, func: ", layer.activation)
        else:
            print(f"Layer-{i + 1}: {number_of_units} units, func: None")
            

def get_best_model(tuner):
    '''
    Describe and return the best model found from keras tuner
    '''
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)

    describe_model(best_model)

    print("\nOther params:")
    ignore_params = ["tuner", "activation", "layer"]
    for param, value in best_hps.values.items():
        if not any(word in param for word in ignore_params):
            print(f"{param}: {value}")

    return best_model

### 3.2. Model with 3 layers

In [10]:
def model_3l_builder(hp):
    model = Sequential()
    model.add(Dense(68, input_dim = 68, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(3, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [11]:
tuner_3l = kt.Hyperband(
    model_3l_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo'
)
tuner_3l.search(x_train, y_train, epochs=10, callbacks=[stop_early])

Trial 30 Complete [00h 00m 55s]
accuracy: 0.9989480972290039

Best accuracy So Far: 0.9990357756614685
Total elapsed time: 00h 11m 35s
INFO:tensorflow:Oracle triggered exit


In [12]:
model_3l = get_best_model(tuner_3l)
model_3l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x17f4a2310>
Layer-2: 32 units, func:  <function tanh at 0x17f4a2670>
Layer-3: 3 units, func:  <function softmax at 0x17f49d8b0>
learning_rate: 0.001
Epoch 1/100


2022-11-22 22:34:40.154500: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2282/2282 [==============================] - ETA: 0s - loss: 0.0489 - accuracy: 0.9877

2022-11-22 22:35:00.478855: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2282/2282 [==============================] - 24s 10ms/step - loss: 0.0489 - accuracy: 0.9877 - val_loss: 0.0067 - val_accuracy: 0.9986
Epoch 2/100
2282/2282 [==============================] - 22s 10ms/step - loss: 0.0096 - accuracy: 0.9975 - val_loss: 0.0045 - val_accuracy: 0.9989
Epoch 3/100
2282/2282 [==============================] - 22s 10ms/step - loss: 0.0100 - accuracy: 0.9976 - val_loss: 0.0076 - val_accuracy: 0.9979
Epoch 4/100
2282/2282 [==============================] - 22s 10ms/step - loss: 0.0063 - accuracy: 0.9982 - val_loss: 0.0038 - val_accuracy: 0.9986
Epoch 5/100
2282/2282 [==============================] - 22s 10ms/step - loss: 0.0066 - accuracy: 0.9981 - val_loss: 0.0012 - val_accuracy: 0.9996
Epoch 6/100
2282/2282 [==============================] - 22s 10ms/step - loss: 0.0058 - accuracy: 0.9985 - val_loss: 0.0014 - val_accuracy: 0.9993
Epoch 7/100
2282/2282 [==============================] - 22s 10ms/step - loss: 0.0073 - accuracy: 0.9981 - val_loss: 0.0034 - val_

In [13]:
final_models["3_layers"] = model_3l

### 3.3. Model with 5 layers

In [14]:
def model_5l_builder(hp):
    model = Sequential()
    model.add(Dense(68, input_dim = 68, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(3, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [15]:
tuner_5l = kt.Hyperband(
    model_5l_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_1'
)
tuner_5l.search(x_train, y_train, epochs=10, callbacks=[stop_early])

Trial 30 Complete [00h 01m 07s]
accuracy: 0.9977647066116333

Best accuracy So Far: 0.9995178580284119
Total elapsed time: 00h 14m 00s
INFO:tensorflow:Oracle triggered exit


In [17]:
model_5l = get_best_model(tuner_5l)
model_5l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x17f4a2310>
Layer-2: 96 units, func:  <function relu at 0x17f4a2310>
Layer-3: 416 units, func:  <function relu at 0x17f4a2310>
Layer-4: 320 units, func:  <function relu at 0x17f4a2310>
Layer-5: 3 units, func:  <function softmax at 0x17f49d8b0>
learning_rate: 0.0001
Epoch 1/100
   1/2282 [..............................] - ETA: 12:30 - loss: 1.1718 - accuracy: 0.3000

2022-11-22 23:03:14.751713: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2282/2282 [==============================] - ETA: 0s - loss: 0.0861 - accuracy: 0.9754

2022-11-22 23:03:38.016530: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2282/2282 [==============================] - 28s 12ms/step - loss: 0.0861 - accuracy: 0.9754 - val_loss: 0.0077 - val_accuracy: 0.9988
Epoch 2/100
2282/2282 [==============================] - 28s 12ms/step - loss: 0.0099 - accuracy: 0.9979 - val_loss: 0.0114 - val_accuracy: 0.9977
Epoch 3/100
2282/2282 [==============================] - 26s 12ms/step - loss: 0.0077 - accuracy: 0.9978 - val_loss: 0.0027 - val_accuracy: 0.9993
Epoch 4/100
2282/2282 [==============================] - 26s 11ms/step - loss: 0.0066 - accuracy: 0.9982 - val_loss: 0.0021 - val_accuracy: 0.9993
Epoch 5/100
2282/2282 [==============================] - 27s 12ms/step - loss: 0.0046 - accuracy: 0.9990 - val_loss: 9.3464e-04 - val_accuracy: 0.9996
Epoch 6/100
2282/2282 [==============================] - 27s 12ms/step - loss: 0.0051 - accuracy: 0.9986 - val_loss: 0.0013 - val_accuracy: 0.9995
Epoch 7/100
2282/2282 [==============================] - 27s 12ms/step - loss: 0.0045 - accuracy: 0.9989 - val_loss: 0.0021 - 

In [18]:
final_models["5_layers"] = model_5l

### 3.4. Model with 7 layers (along with Dropout)

In [23]:
def model_7lD_builder(hp):
    model = Sequential()
    model.add(Dense(68, input_dim = 68, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_dropout_1 = hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)
    hp_dropout_2 = hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_1))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_2))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(3, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [39]:
tuner_7lD = kt.Hyperband(
    model_7lD_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_4'
)
tuner_7lD.search(x_train, y_train, epochs=10, callbacks=[stop_early])

INFO:tensorflow:Reloading Oracle from existing project keras_tuner_dir/keras_tuner_demo_4/oracle.json
INFO:tensorflow:Reloading Tuner from keras_tuner_dir/keras_tuner_demo_4/tuner0.json
INFO:tensorflow:Oracle triggered exit


In [41]:
model_7lD = get_best_model(tuner_7lD)
model_7lD.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x17f4a2310>
Layer-2: 480 units, func:  <function relu at 0x17f4a2310>
Layer-3: 0 units, func: None
Layer-4: 256 units, func:  <function relu at 0x17f4a2310>
Layer-5: 0 units, func: None
Layer-6: 480 units, func:  <function relu at 0x17f4a2310>
Layer-7: 3 units, func:  <function softmax at 0x17f49d8b0>

Other params:
dropout_1: 0.30000000000000004
dropout_2: 0.4
learning_rate: 0.0001
Epoch 1/100


2022-11-22 23:59:17.522675: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2282/2282 [==============================] - ETA: 0s - loss: 0.0917 - accuracy: 0.9722

2022-11-22 23:59:44.220298: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2282/2282 [==============================] - 31s 13ms/step - loss: 0.0917 - accuracy: 0.9722 - val_loss: 0.0082 - val_accuracy: 0.9984
Epoch 2/100
2282/2282 [==============================] - 31s 14ms/step - loss: 0.0121 - accuracy: 0.9968 - val_loss: 0.0067 - val_accuracy: 0.9988
Epoch 3/100
2282/2282 [==============================] - 29s 13ms/step - loss: 0.0096 - accuracy: 0.9975 - val_loss: 0.0134 - val_accuracy: 0.9965
Epoch 4/100
2282/2282 [==============================] - 30s 13ms/step - loss: 0.0083 - accuracy: 0.9981 - val_loss: 0.0051 - val_accuracy: 0.9989
Epoch 5/100
2282/2282 [==============================] - 30s 13ms/step - loss: 0.0060 - accuracy: 0.9986 - val_loss: 0.0025 - val_accuracy: 0.9996
Epoch 6/100
2282/2282 [==============================] - 30s 13ms/step - loss: 0.0072 - accuracy: 0.9983 - val_loss: 0.0046 - val_accuracy: 0.9995
Epoch 7/100
2282/2282 [==============================] - 30s 13ms/step - loss: 0.0047 - accuracy: 0.9990 - val_loss: 0.0024 - val_

In [42]:
final_models["7_layers_with_dropout"] = model_7lD

### 3.5. Model with 7 layers

In [44]:
def model_7l_builder(hp):
    model = Sequential()
    model.add(Dense(68, input_dim = 68, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_layer_4 = hp.Int('layer_4', min_value=32, max_value=512, step=32)
    hp_layer_5 = hp.Int('layer_5', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(units=hp_layer_4, activation=hp_activation))
    model.add(Dense(units=hp_layer_5, activation=hp_activation))
    model.add(Dense(3, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [46]:
tuner_7l = kt.Hyperband(
    model_7l_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_5'
)
tuner_7l.search(x_train, y_train, epochs=10, callbacks=[stop_early])

Trial 30 Complete [00h 01m 26s]
accuracy: 0.9993863701820374

Best accuracy So Far: 0.9993863701820374
Total elapsed time: 00h 17m 16s
INFO:tensorflow:Oracle triggered exit


In [47]:
model_7l = get_best_model(tuner_7l)
model_7l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x17f4a2310>
Layer-2: 160 units, func:  <function relu at 0x17f4a2310>
Layer-3: 416 units, func:  <function relu at 0x17f4a2310>
Layer-4: 288 units, func:  <function relu at 0x17f4a2310>
Layer-5: 512 units, func:  <function relu at 0x17f4a2310>
Layer-6: 256 units, func:  <function relu at 0x17f4a2310>
Layer-7: 3 units, func:  <function softmax at 0x17f49d8b0>

Other params:
learning_rate: 0.0001
Epoch 1/100


2022-11-23 00:37:07.916975: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2282/2282 [==============================] - ETA: 0s - loss: 0.0573 - accuracy: 0.9844

2022-11-23 00:37:36.882232: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2282/2282 [==============================] - 34s 15ms/step - loss: 0.0573 - accuracy: 0.9844 - val_loss: 0.0053 - val_accuracy: 0.9982
Epoch 2/100
2282/2282 [==============================] - 33s 15ms/step - loss: 0.0104 - accuracy: 0.9974 - val_loss: 0.0030 - val_accuracy: 0.9991
Epoch 3/100
2282/2282 [==============================] - 35s 15ms/step - loss: 0.0089 - accuracy: 0.9979 - val_loss: 0.0014 - val_accuracy: 0.9996
Epoch 4/100
2282/2282 [==============================] - 34s 15ms/step - loss: 0.0090 - accuracy: 0.9979 - val_loss: 0.0016 - val_accuracy: 0.9995
Epoch 5/100
2282/2282 [==============================] - 34s 15ms/step - loss: 0.0067 - accuracy: 0.9983 - val_loss: 0.0011 - val_accuracy: 0.9998
Epoch 6/100
2282/2282 [==============================] - 34s 15ms/step - loss: 0.0043 - accuracy: 0.9990 - val_loss: 0.0036 - val_accuracy: 0.9989
Epoch 7/100
2282/2282 [==============================] - 34s 15ms/step - loss: 0.0053 - accuracy: 0.9988 - val_loss: 0.0059 - val_

In [48]:
final_models["7_layers"] = model_7l

### 3.6. Describe Final Models

In [11]:
for name, model in final_models.items():
    print(f"{name}: ", end="")
    describe_model(model)
    print()

3_layers: Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x179e05b80>
Layer-2: 32 units, func:  <function tanh at 0x179e05ee0>
Layer-3: 3 units, func:  <function softmax at 0x179e05160>

5_layers: Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x179e05b80>
Layer-2: 96 units, func:  <function relu at 0x179e05b80>
Layer-3: 416 units, func:  <function relu at 0x179e05b80>
Layer-4: 320 units, func:  <function relu at 0x179e05b80>
Layer-5: 3 units, func:  <function softmax at 0x179e05160>

7_layers_with_dropout: Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x179e05b80>
Layer-2: 480 units, func:  <function relu at 0x179e05b80>
Layer-3: 0 units, func: None
Layer-4: 256 units, func:  <function relu at 0x179e05b80>
Layer-5: 0 units, func: None
Layer-6: 480 units, func:  <function relu at 0x179e05b80>
Layer-7: 3 units, func:  <function softmax at 0x179e05160>

7_layers: Describe models architecture
Layer-1: 68 units,

## 4. Evaluate model with Train set

In [26]:
train_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(x_test, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(y_test, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1, 2])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1, 2])
    
    train_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

train_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(train_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,3_layers,"[1.0, 1.0, 0.999]","[0.999, 1.0, 1.0]","[0.999, 1.0, 0.999]","[[1913, 0, 2], [0, 1844, 0], [0, 0, 1945]]"
1,5_layers,"[0.999, 1.0, 0.999]","[0.999, 1.0, 0.999]","[0.999, 1.0, 0.999]","[[1914, 0, 1], [0, 1844, 0], [1, 0, 1944]]"
2,7_layers_with_dropout,"[0.999, 1.0, 0.999]","[0.999, 1.0, 0.999]","[0.999, 1.0, 0.999]","[[1914, 0, 1], [0, 1844, 0], [1, 0, 1944]]"
3,7_layers,"[0.999, 1.0, 0.999]","[0.999, 1.0, 0.999]","[0.999, 1.0, 0.999]","[[1914, 0, 1], [0, 1844, 0], [1, 0, 1944]]"


## 5. Evaluation with test set

In [22]:
# load dataset
test_df = pd.read_csv("./test.csv")

# Categorizing label
test_df.loc[test_df["label"] == "C", "label"] = 0
test_df.loc[test_df["label"] == "H", "label"] = 1
test_df.loc[test_df["label"] == "L", "label"] = 2

print(f'Number of rows: {test_df.shape[0]} \nNumber of columns: {test_df.shape[1]}\n')
print(f"Labels: \n{test_df['label'].value_counts()}\n")

Number of rows: 710 
Number of columns: 69

Labels: 
1    241
2    235
0    234
Name: label, dtype: int64



In [23]:
# Standard Scaling of features
test_x = test_df.drop("label", axis = 1)
test_x = pd.DataFrame(sc.transform(test_x))

test_y = test_df["label"]

# # Converting prediction to categorical
test_y_cat = to_categorical(test_y)

In [27]:
test_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(test_x, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(test_y_cat, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1, 2])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1, 2])
    
    test_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

test_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(test_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,7_layers_with_dropout,"[0.983, 1.0, 1.0]","[1.0, 0.992, 0.991]","[0.992, 0.996, 0.996]","[[234, 0, 0], [2, 239, 0], [2, 0, 233]]"
1,5_layers,"[0.984, 0.937, 0.883]","[0.803, 0.992, 0.991]","[0.885, 0.964, 0.934]","[[188, 16, 30], [1, 239, 1], [2, 0, 233]]"
2,7_layers,"[0.984, 0.825, 1.0]","[0.782, 0.996, 0.991]","[0.871, 0.902, 0.996]","[[183, 51, 0], [1, 240, 0], [2, 0, 233]]"
3,3_layers,"[0.88, 0.732, 1.0]","[0.624, 0.996, 0.919]","[0.73, 0.844, 0.958]","[[146, 88, 0], [1, 240, 0], [19, 0, 216]]"


## 6. Conclusion & Dumped model

Best model according to evaluation is model with 7 layers include some dropout layers

In [54]:
# Dump the best model to a pickle file
with open("./model/plank_dp.pkl", "wb") as f:
    pickle.dump(final_models["7_layers_with_dropout"], f)

INFO:tensorflow:Assets written to: ram://72fd27ad-5e1d-4761-a5f0-b2b4682c6a65/assets


In [55]:
# Dump final results
with open("./model/all_dp.pkl", "wb") as f:
    pickle.dump(final_models, f)

INFO:tensorflow:Assets written to: ram://cc39563d-8dfe-4f23-9dc6-71462ea5ef5b/assets
INFO:tensorflow:Assets written to: ram://5855831e-e2d4-4bb1-9799-a65f926cefb3/assets
INFO:tensorflow:Assets written to: ram://bf30ae68-3e13-4624-8b29-c9c341693ed8/assets
INFO:tensorflow:Assets written to: ram://9b787241-d173-4542-88d3-5551c1cbb80d/assets
